In [384]:

import sys
import pandas as pd 
import tensorflow as tf 
# featuretools for automated feature engineering
import featuretools as ft
import featuretools.variable_types as vtypes
 

In [385]:
df_Product_org = pd.read_csv('./data/P4.csv')
df_Tool_org = pd.read_csv('./data/M4.csv')

In [386]:
df_Product = df_Product_org.copy(deep=False)
df_Tool = df_Tool_org.copy(deep=False)

In [389]:
df_Product['PPkey'] = df_Product['MFG_DATE'].astype(str)+'_' +df_Product['TOOLG_ID'] +'_' +df_Product['PROD_ID'] 
df_Product['PRkey'] = df_Product['MFG_DATE'].astype(str)+'_' +df_Product['TOOLG_ID'] 
df_Tool['TRkey'] = df_Tool['MFG_DATE'].astype(str)+'_' +df_Tool['TOOLG_ID'] 
 
df_Product['dtMFG_DATE'] = df_Product['MFG_DATE'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
df_Tool['dtMFG_DATE'] = df_Tool['MFG_DATE'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))


In [388]:
# es = ft.EntitySet(id = 'MRATIO')
df_Product

,PROD_ID,MOVE_P,MFG_DATE,TOOLG_ID,ProdWIP,ProdHoldWIP,ProdMove,PRkey,dtMFG_DATE
0,D38BR01A,3.216123,20200126,PW_PIX,0.0,0.0,7500.0,20200126_PW_PIX,2020-01-26
1,D38CL01A,3.216123,20200126,PW_PIX,25.0,25.0,7500.0,20200126_PW_PIX,2020-01-26
2,D45AL03A,1.072041,20200126,PW_PIX,0.0,0.0,2500.0,20200126_PW_PIX,2020-01-26
3,F40AL02B,1.072041,20200126,PW_PIX,25.0,0.0,2500.0,20200126_PW_PIX,2020-01-26
4,25D_M81,3.428180,20200128,PW_PIX,0.0,0.0,10000.0,20200128_PW_PIX,2020-01-28
...,...,...,...,...,...,...,...,...,...
166555,L11GD12A,1.519893,20200303,PK_DUVKrF,172.0,0.0,57800.0,20200303_PK_DUVKrF,2020-03-03
166556,L11GE16A,0.339215,20200303,PK_DUVKrF,50.0,0.0,12900.0,20200303_PK_DUVKrF,2020-03-03
166557,L11GE39A,2.687423,20200303,PK_DUVKrF,298.0,150.0,102200.0,20200303_PK_DUVKrF,2020-03-03
166558,L13DY47A,0.015777,20200303,PK_DUVKrF,6.0,2.0,600.0,20200303_PK_DUVKrF,2020-03-03


In [392]:
# 創建一個空的實體集
 
es = ft.EntitySet(id = 'MRATIO')
#Entityset 把多個實體Entity進行合併
# Product
es = es.entity_from_dataframe(entity_id = 'Product',
             dataframe = df_Product,  
            #  variable_types={'MFG_DATE': ft.variable_types.Categorical},
            make_index = False, 
            # index="ID",
            # index = 'PRkey',
            index ='PPkey',
            time_index = 'dtMFG_DATE',
            variable_types={'MFG_DATE':vtypes.Categorical},

             )
# Tool
es = es.entity_from_dataframe(entity_id = 'Machine',
             dataframe = df_Tool,  
            #  variable_types={'MFG_DATE': ft.variable_types.Categorical},
            # make_index = True, 
            # index="ID",
            index = 'TRkey', 
            time_index = 'dtMFG_DATE',
            variable_types={'MFG_DATE':vtypes.Categorical},
            )   





In [393]:
print(es["Product"].variables)
print(es["Machine"].variables)

[<Variable: PPkey (dtype = index)>, <Variable: PROD_ID (dtype = categorical)>, <Variable: MOVE_P (dtype = numeric)>, <Variable: TOOLG_ID (dtype = categorical)>, <Variable: ProdWIP (dtype = numeric)>, <Variable: ProdHoldWIP (dtype = numeric)>, <Variable: ProdMove (dtype = numeric)>, <Variable: PRkey (dtype = categorical)>, <Variable: dtMFG_DATE (dtype: datetime_time_index, format: None)>, <Variable: MFG_DATE (dtype = categorical)>]
[<Variable: TRkey (dtype = index)>, <Variable: TOOLG_ID (dtype = categorical)>, <Variable: IS_HOLIDAY (dtype = numeric)>, <Variable: M_NUM (dtype = numeric)>, <Variable: UP_TIME (dtype = numeric)>, <Variable: C_UP_TIME (dtype = numeric)>, <Variable: LOT_SIZE (dtype = numeric)>, <Variable: C_LOT_SIZE (dtype = numeric)>, <Variable: EQP_UTIL (dtype = numeric)>, <Variable: C_EQP_UTIL (dtype = numeric)>, <Variable: EQP_AVAIL_RATE (dtype = numeric)>, <Variable: TC (dtype = numeric)>, <Variable: PROCESS_TIME (dtype = numeric)>, <Variable: PROCESS_JOBTIME (dtype = nu

In [ ]:
# from featuretools.variable_types import list_variable_types
# list_variable_types()
# from featuretools.variable_types import graph_variable_types
# graph_variable_types()

In [394]:
r= ft.Relationship(es['Machine']['TRkey'],es['Product']['PRkey'])
es = es.add_relationship(r)

In [ ]:
# default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
# default_trans_primitives =  ["day", "year", "month", "weekday","num_words","haversine","num_characters"]


In [ ]:
# target 為Product 沒有意義
# default_agg_primitives=["std"]
# default_trans_primietives = []

# feature_matrix,feature_names= ft.dfs( entityset = es,
#                 target_entity='Product',
#                 trans_primitives = default_trans_primietives,
#                 agg_primitives=default_agg_primitives,
#                 primitive_options={ 
#                     #'std': {'ignore_entities': ['cohorts', 'log']}, 
#                     'std': {'include_variables': 
#                                 {'Product': ['ProdMove', 'MOVE_P'],
#                                 'Machine': ['M_NUM'],
#                                 }},

#                 },

#             max_depth=2,
#             features_only=False,verbose=True) 

In [395]:
# Default primitives from featuretools
#default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
# default_trans_primitives =  ["day", "year", "month", "weekday", "haversine",   "characters"]
#default_trans_primitives =  ["day", "year", "month", "weekday"]

default_trans_primitives = ["weekday"]
default_agg_primitives=["std"]
feature_matrix2,feature_names2= ft.dfs( entityset = es,
                target_entity='Machine',
                trans_primitives = default_trans_primitives, #對母表 單一筆資料去處理
                agg_primitives=default_agg_primitives, #對 子表做分析
                primitive_options={ 
                    #'std': {'ignore_entities': ['cohorts', 'log']}, 
                    'std': {'include_variables': 
                                {'Product': ['ProdMove', 'MOVE_P'],#子表(1:N的表去計算)
                                'Machine': ['M_NUM'], ## 沒有用因為 要針對子表(1:N的表去計算) 
                                }},
                },
            max_depth=2,
            features_only=False,verbose=True) 

Built 43 features
Elapsed: 00:00 | Progress: 100%|██████████


In [397]:
#驗證資料
print(feature_matrix2[['MFG_DATE','WEEKDAY(dtMFG_DATE)','TOOLG_ID','STD(Product.MOVE_P)','STD(Product.ProdMove)']].head(1))
print(df_Product[df_Product['PRkey']=='20200122_MA_Alps'].std())
 
# df_Product[(df_Product.PROD_ID=='4GDDR4_25A') & (df_Product.MFG_DATE==20200122)].std()

                  MFG_DATE  WEEKDAY(dtMFG_DATE) TOOLG_ID  STD(Product.MOVE_P)  \
TRkey                                                                           
20200122_MA_Alps  20200122                    2  MA_Alps             7.968777   

                  STD(Product.ProdMove)  
TRkey                                    
20200122_MA_Alps            5952.676623  
MOVE_P                7.968777
MFG_DATE                   0.0
ProdWIP               6.933752
ProdHoldWIP                0.0
ProdMove           5952.676623
dtMFG_DATE     0 days 00:00:00
dtype: object


In [ ]:
#feature_matrix2

In [404]:
# # Feature Selection

# # Threshold for removing correlated variables
# threshold = 0.95

# # Absolute value correlation matrix
# corr_matrix = feature_names2.corr().abs()
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# upper.head(50)

In [402]:

#select features that have at least 2 unique values and that are not all null
#(feature_matrix, features=None):
from featuretools.selection import remove_low_information_features  ## 移除较小信息的特征

#Removes columns from a feature matrix that have higher than a set threshold of null values.
#(feature_matrix, features=None, pct_null_threshold=0.95):
from featuretools.selection import remove_highly_null_features

#Removes columns in feature matrix where all the values are the same.
#(feature_matrix, features=None, count_nan_as_value=False):
from featuretools.selection import remove_single_value_features

#Removes columns in feature matrix that are highly correlated with another column.
#(feature_matrix, features=None, pct_corr_threshold=0.95,features_to_check=None, features_to_keep=None):
from featuretools.selection import remove_highly_correlated_features 
feature_matrix = feature_matrix2
features_list =feature_names2


print('features',feature_matrix.shape)
features1, feature_names1 = remove_low_information_features(feature_matrix, features_list)
print('被刪除的欄位 remove_low_information_features',feature_matrix.shape)
print('Removed %d features from training features' % (feature_matrix.shape[1] - features1.shape[1])) 
print(set(feature_matrix) - set(features1))

features1, feature_names1 = remove_highly_null_features(features1, feature_names1,0.95)
print('被刪除的欄位 remove_highly_null_features',features1.shape)
print('Removed %d features from training features' % (feature_matrix.shape[1] - features1.shape[1])) 
print(set(feature_matrix) - set(features1))

features1, feature_names1 = remove_single_value_features(features1, feature_names1)
print('被刪除的欄位 remove_single_value_features',features1.shape)
print('Removed %d features from training features' % (feature_matrix.shape[1] - features1.shape[1])) 
print(set(feature_matrix) - set(features1))

features1, feature_names1 = remove_highly_correlated_features(features1, feature_names1,0.95)
print('被刪除的欄位 remove_highly_correlated_features',features1.shape)
print('Removed %d features from training features' % (feature_matrix.shape[1] - features1.shape[1])) 
print(set(feature_matrix) - set(features1))

features (1647, 43)
被刪除的欄位 remove_low_information_features (1647, 43)
Removed 6 features from training features
{'BATCH_SIZE', 'C_TOOLG_LOADING', 'SAMPLING_RATE', 'MANAGEMENT_WIP_QTY', 'DISPATCHING', 'C_TOOL_LOADING'}
被刪除的欄位 remove_highly_null_features (1647, 37)
Removed 6 features from training features
{'BATCH_SIZE', 'C_TOOLG_LOADING', 'SAMPLING_RATE', 'MANAGEMENT_WIP_QTY', 'DISPATCHING', 'C_TOOL_LOADING'}
被刪除的欄位 remove_single_value_features (1647, 37)
Removed 6 features from training features
{'BATCH_SIZE', 'C_TOOLG_LOADING', 'SAMPLING_RATE', 'MANAGEMENT_WIP_QTY', 'DISPATCHING', 'C_TOOL_LOADING'}
被刪除的欄位 remove_highly_correlated_features (1647, 29)
Removed 14 features from training features
{'C_LOT_SIZE', 'BATCH_SIZE', 'C_TOOLG_LOADING', 'RUN_WIP', 'INLINE_CT_BY_WAFER', 'SAMPLING_RATE', 'ARRIVAL_WIP_QTY', 'MANAGEMENT_WIP_QTY', 'DISPATCHING', 'NO_HOLD_QTY', 'C_TOOL_LOADING', 'STD(Product.ProdMove)', 'CLOSE_WIP_QTY', 'NUM_RECIPE'}


In [405]:
feature_matrix.columns

Index(['TOOLG_ID', 'IS_HOLIDAY', 'M_NUM', 'UP_TIME', 'C_UP_TIME', 'LOT_SIZE',
       'C_LOT_SIZE', 'EQP_UTIL', 'C_EQP_UTIL', 'EQP_AVAIL_RATE', 'TC',
       'PROCESS_TIME', 'PROCESS_JOBTIME', 'INLINE_CT_BY_LOT',
       'INLINE_CT_BY_WAFER', 'WIP_QTY', 'NO_HOLD_QTY', 'MOVE_QTY',
       'ARRIVAL_WIP_QTY', 'RUN_WIP', 'RUN_WIP_RATIO', 'CLOSE_WIP_QTY',
       'MANAGEMENT_WIP_QTY', 'C_TC', 'C_CLOSE_WIP', 'C_TOOLG_LOADING',
       'C_TOOL_LOADING', 'DISPATCHING', 'HOLD_RATE', 'ENG_LOT_RATE',
       'HOT_LOT_RATE', 'BACKUP_BY_RATE', 'BACKUP_FOR_RATE', 'REWORK_LOT_RATE',
       'QUE_LOT_RATE', 'SAMPLING_RATE', 'NUM_RECIPE', 'CHANGE_RECIPE',
       'BATCH_SIZE', 'MFG_DATE', 'STD(Product.MOVE_P)',
       'STD(Product.ProdMove)', 'WEEKDAY(dtMFG_DATE)'],
      dtype='object')

In [403]:

# drop_cols=['MFG_DATE','PPkey','PRkey','TRkey','dtMFG_DATE','dtMFG_DATE']
drop_cols=['MFG_DATE''dtMFG_DATE','dtMFG_DATE']
target_cols=['TRCT']
 
df = feature_matrix.drop(drop_cols, axis=1)

,TOOLG_ID,IS_HOLIDAY,M_NUM,UP_TIME,C_UP_TIME,LOT_SIZE,C_LOT_SIZE,EQP_UTIL,C_EQP_UTIL,EQP_AVAIL_RATE,...,REWORK_LOT_RATE,QUE_LOT_RATE,SAMPLING_RATE,NUM_RECIPE,CHANGE_RECIPE,BATCH_SIZE,MFG_DATE,STD(Product.MOVE_P),STD(Product.ProdMove),WEEKDAY(dtMFG_DATE)
TRkey,,,,,,,,,,,,,,,,,,,,,
20200122_MA_Alps,MA_Alps,0.9499,4.0,0.619437,0.206197,24.096774,0.196026,0.660880,0.504139,0.498691,...,0.000000,0.609103,0,8,0.419355,NaN,20200122,7.968777,5952.676623,2
20200122_PK_DUVKrF,PK_DUVKrF,0.9499,30.0,0.978936,0.130694,21.543705,0.220839,0.960454,0.346952,0.063863,...,0.027075,0.294861,0,579,0.677845,NaN,20200122,1.146339,44922.744351,2
20200122_PW_PIX,PW_PIX,0.9499,3.0,1.000000,0.126728,21.944444,0.214632,0.656590,0.504476,0.587963,...,0.008228,0.063291,0,23,0.805556,NaN,20200122,3.196530,5050.517287,2
20200122_XE_Sorter,XE_Sorter,0.9499,15.0,0.971326,0.125776,22.515499,0.202504,0.668134,0.495682,0.863297,...,0.020381,0.145393,0,2,0.299079,NaN,20200122,0.907129,30309.888553,2
20200123_MA_Alps,MA_Alps,1.0000,4.0,0.758362,0.161453,23.275862,0.187678,0.452553,0.732736,0.685629,...,0.000000,0.776296,0,12,0.586207,NaN,20200123,4.565021,3081.389027,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20210307_XE_Sorter,XE_Sorter,1.0000,16.0,0.994306,0.127505,22.847264,0.186647,0.634794,0.517918,0.907057,...,0.012771,0.250008,0,2,0.312062,NaN,20210307,0.827411,24877.759005,6
20210308_MA_Alps,MA_Alps,0.9499,6.0,0.959960,0.132039,20.035714,0.212846,0.620041,0.530249,0.462821,...,0.085561,0.677362,0,13,0.607143,NaN,20210308,3.778364,2078.100294,0
20210308_PK_DUVKrF,PK_DUVKrF,0.9499,32.0,0.976119,0.129871,22.090909,0.193052,0.984803,0.333859,0.067614,...,0.024113,0.475008,0,436,0.659951,NaN,20210308,1.346584,60535.663281,0


# 客制功能

In [ ]:
from featuretools.primitives import make_agg_primitive, make_trans_primitive
from featuretools.variable_types import Text, Numeric,Boolean,Variable

In [ ]:
# custom function so the name of the feature prints out correctly
def make_name(self):
    return "%s_goal_last_%d" % (self.kwargs['Qty1'], self.kwargs['Qty2'])

In [ ]:
 CompareMove

In [ ]:
def compare_Qty(Qty1,Qty2):
   return Qty1>Qty2
CompareMove = make_trans_primitive(function=compare_Qty,
                                          input_types=[Numeric, Numeric],
                                          return_type=Boolean,
                                          description="compare_Qty"
                                          #cls_attributes={"generate_name": make_name, "uses_full_entity":True}
                                          )

input_vars = [es["Machine"]["MOVE_QTY"], es["Machine"]["WIP_QTY"]]


# Compare_Move = CompareMove(*input_vars)
# #Compare_Move = CompareMove(Qty1=es["Machine"]["MOVE_QTY"], Qty2=es["Machine"]["WIP_QTY"])

# features = [Compare_Move]


# fm = ft.calculate_feature_matrix(entityset=es, features=features) 


feature_matrix, features = ft.dfs(entityset=es,target_entity="Product",agg_primitives=[],
                        trans_primitives=[CompareMove],
                         primitive_options={ 
                            #'std': {'ignore_entities': ['cohorts', 'log']}, 
                        'CompareMove': {'include_variables': 
                                        {'Product': ['ProdMove', 'MOVE_P'],
                                        }},

                        },
                        max_depth=2)

In [ ]:
feature_matrix

In [ ]:
# Feature lineage graphs
# Understanding Feature Output¶
# In general, Featuretools references generated features through the feature name. In order to make features easier to understand, Featuretools offers two additional tools, featuretools.graph_feature() and featuretools.describe_feature(), to help explain what a feature is and the steps Featuretools took to generate it.
feature = feature_names[54]
ft.graph_feature(feature)

In [ ]:
ft.describe_feature(feature)

In [ ]:
feature_matrix,feature_names =ft.dfs(entityset = es, target_entity = 'WIP',
                       trans_primitives=['weekday', 'day'],
                       #agg_primitives=default_agg_primitives, 
                        verbose = 1,
                       features_only = False)


 

In [ ]:
#累績總計 與總數
 
feature_matrix, features_list = ft.dfs(entityset=es, 
    verbose = 1,
	target_entity='WIP',
    agg_primitives=[], 
	trans_primitives=['cum_sum', 'cum_count'], 
	groupby_trans_primitives=['cum_sum', 'cum_count'], #累積總計、累積總數
    
    ignore_variables={'WIP': ['PRIORITY']}
    
	)
feature_matrix    


In [ ]:
from featuretools.primitives import CumCount
#累績總數
cum_count = CumCount()
cum_count([1, 2, 3, 4, None, 5]).tolist()


In [ ]:
from featuretools.primitives import CumCount
from featuretools.primitives import IsWeekend
#累績總數
cum_count = CumCount()
cum_count([1, 2, 3, 4, None, 5]).tolist()
isWkend = IsWeekend()
feature_matrix, features_list = ft.dfs(entityset=es, 
    verbose = 1,
	target_entity='WIP',
    agg_primitives=[], 
	trans_primitives=['cum_sum','cum_count','is_weekend'], 
	primitive_options={
        'cum_sum': {
                'include_variables':
                    {'WIP': ['WIP_QTY']},
                'ignore_variables':
                    {'WIP': ['IS_MAIN_ROUTE']}    
                    
                    },
		'cum_count': {
                'include_variables':
                    {'WIP': ['WIP_QTY']},
                'ignore_variables':
                    {'WIP': ['IS_MAIN_ROUTE']}    
                    
                    },
        'is_weekend': {
                'ignore_variables':
                        {'WIP': ['ACTUAL_WP_OUT']}}
        }
    
	)



feature_matrix    

AttributeError: 'list' object has no attribute 'corr'

In [ ]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(train_X, train_y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(train_X)
X_selected_df = pd.DataFrame(X_new, columns=[train_X.columns[i] for i in range(len(train_X.columns)) if model.get_support()[i]])
X_selected_df.shape

In [ ]:
random_forest = RandomForestClassifier(n_estimators=2000,oob_score=True)
random_forest.fit(X_selected_df, train_y)

In [ ]:
## 客制function

In [ ]:
'''
from featuretools.primitives import make_agg_primitive, make_trans_primitive
from featuretools.variable_types import Text, Numeric

def avg_goals_previous_n_games(home_team, away_team, home_goals, away_goals, which_team=None, n=1):
    # make dataframe so it's easier to work with
    df = pd.DataFrame({
        "home_team": home_team,
        "away_team": away_team,
        "home_goals": home_goals,
        "away_goals": away_goals
        })

    result = []
    for i, current_game in df.iterrows():
        # get the right team for this game
        team = current_game[which_team]

        # find all previous games that have been played
        prev_games =  df.iloc[:i]

        # only get games the team participated in
        participated = prev_games[(prev_games["home_team"] == team) | (prev_games["away_team"] == team)]
        if participated.shape[0] < n:
            result.append(None)
            continue

        # get last n games
        last_n = participated.tail(n)

        # calculate games per game
        goal_as_home = (last_n["home_team"] == team) * last_n["home_goals"]
        goal_as_away = (last_n["away_team"] == team) * last_n["away_goals"]

        # calculate mean across all home and away games
        mean = (goal_as_home + goal_as_away).mean()

        result.append(mean)

    return result
# custom function so the name of the feature prints out correctly
def make_name(self):
    return "%s_goal_last_%d" % (self.kwargs['which_team'], self.kwargs['n'])
AvgGoalPreviousNGames = make_trans_primitive(function=avg_goals_previous_n_games,
                                          input_types=[Categorical, Categorical, Numeric, Numeric],
                                          return_type=Numeric,
                                          cls_attributes={"generate_name": make_name, "uses_full_entity":True})

input_vars = [es["matches"]["home_team"], es["matches"]["away_team"], es["matches"]["home_goals"], es["matches"]["away_goals"]]
home_team_last1 = AvgGoalPreviousNGames(*input_vars, which_team="home_team", n=1)
home_team_last3 = AvgGoalPreviousNGames(*input_vars, which_team="home_team", n=3)
home_team_last5 = AvgGoalPreviousNGames(*input_vars, which_team="home_team", n=5)
away_team_last1 = AvgGoalPreviousNGames(*input_vars, which_team="away_team", n=1)
away_team_last3 = AvgGoalPreviousNGames(*input_vars, which_team="away_team", n=3)
away_team_last5 = AvgGoalPreviousNGames(*input_vars, which_team="away_team", n=5)

features = [home_team_last1, home_team_last3, home_team_last5,
            away_team_last1, away_team_last3, away_team_last5]


fm = ft.calculate_feature_matrix(entityset=es, features=features) 

fm, feature_defs = ft.dfs(entityset=es, 
                          target_entity="matches",
                          seed_features=features, 
                          agg_primitives=[], 
                          trans_primitives=["day", "month", "year", "weekday", "percentile"])

#https://stackoverflow.com/questions/53579465/how-to-use-featuretools-to-create-features-from-multiple-columns-in-single-dataf                          